In [ ]:
pip install litellm

In [ ]:
pip install smolagents

In [44]:
from huggingface_hub import login

login()


In [ ]:
from litellm import completion
import os

os.environ['GEMINI_API_KEY'] = ""
response = completion(
    model="gemini/gemini-2.0-flash",
    messages=[{"role": "user", "content": "write code for saying hi from LiteLLM"}]
)

In [ ]:
response

In [ ]:
from smolagents import LiteLLMModel

messages = [
  {"role": "user", "content": [{"type": "text", "text": "Hello, how are you?"}]}
]

model = LiteLLMModel("gemini/gemini-1.5-flash", temperature=0.2, max_tokens=1000)
print(model(messages))

In [29]:
from smolagents import Tool
import requests

class AIBolnaCallTool(Tool):
    name = "ai_call_tool"
    description = "Initiates an AI-powered phone call using the Bolna API."

    inputs = {
        "recipient_phone_number": {
            "type": "string",
            "description": "The recipient's phone number in E.164 format (e.g., '+10123456789')."
        },
        "user_data": {
            "type": "object",
            "description": "Optional key-value pairs to pass to the call (e.g., {'variable1': 'value1'}).",
            "default": None,
            "nullable": True  # Allow None as a valid value
        }
    }

    output_type = "string"
    def __init__(self, agent_id , api_token , **kwargs):
        super().__init__(**kwargs)
        self.api_url = "https://api.bolna.dev/call"
        self.agent_id = agent_id
        self.headers = {
            "Authorization": f"Bearer {api_token}",
            "Content-Type": "application/json"
        }


    def forward(self, recipient_phone_number: str, user_data: dict = None):
        """
        Make an AI phone call using the Bolna API.

        Args:
            recipient_phone_number (str): The phone number to call.
            user_data (dict, optional): Additional data to pass to the call.

        Returns:
            str: The API response or an error message.
        """
        # Construct the payload
        payload = {
            "agent_id": self.agent_id,
            "recipient_phone_number": recipient_phone_number
        }
        if user_data:
            payload["user_data"] = user_data

        try:
            # Send the POST request to the Bolna API
            response = requests.post(self.api_url, json=payload, headers=self.headers)
            response.raise_for_status()  # Raise an exception for bad status codes (e.g., 400, 401)

            # Return the API response as text
            print(response.text)
            return response.text

        except requests.RequestException as e:
            return f"Error making AI call: {str(e)}"
        except Exception as e:
            return f"Error processing request: {str(e)}"

In [ ]:
from smolagents import CodeAgent, DuckDuckGoSearchTool
ai_call = AIBolnaCallTool(agent_id = "" , api_token = "")
agent = CodeAgent(tools=[DuckDuckGoSearchTool() , ai_call], model=model)


agent.run("give a call to rika who is software engineer who stated a bisnuss firm in kerala phone number +915452784955")